In [1]:
## We don't want to output too many images to one directory, so we're going to first figure out the length

import subprocess

def getLength(filename):
  result = subprocess.Popen(["ffprobe", filename],
    stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
  return [x for x in result.stdout.readlines() if "Duration" in x]

getLength("/home/ubuntu/SoftRobotRepresentation/Data/recording1.mp4")

['  Duration: 00:17:48.90, start: 0.000000, bitrate: 1196 kb/s\n']